# 01 Data Preparation - Gazelle/Pon Case

**Doel**: 
- Dealers data schoonmaken en Pon-merken identificeren
- Demografie data verwerken (CBS -99997 handling)
- PC4-gemeente mapping en feature engineering
- Demografische clustering (k=5)

**Exports**:
- `data/processed/dealers.parquet`
- `data/processed/demografie.parquet` 
- `outputs/tables/kpis_by_pc4_with_clusters.csv`

In [9]:
# Imports and constants
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

# Paths
RAW = Path("../data/raw")
EXT = Path("../data/external") 
PROC = Path("../data/processed")
OUT = Path("../outputs/tables")

# Ensure directories exist
PROC.mkdir(parents=True, exist_ok=True)
OUT.mkdir(parents=True, exist_ok=True)

# Pon brands definition (from README)
PON_BRANDS = {"gazelle", "cannondale", "union", "kalkhoff", "urban arrow", "cervélo", "cervelo", "focus", "santa cruz"}

print("✅ Setup completed")
print(f"PON brands: {sorted(PON_BRANDS)}")

✅ Setup completed
PON brands: ['cannondale', 'cervelo', 'cervélo', 'focus', 'gazelle', 'kalkhoff', 'santa cruz', 'union', 'urban arrow']


## 1. Dealers Data Processing

In [10]:
# Load dealers data
print("Loading dealers data...")
dealers = pd.read_csv(RAW / "dealer_lijst.csv")
print(f"Raw dealers: {len(dealers):,} records")

# Clean brand names and identify Pon brands
dealers["brand_clean"] = dealers["brand"].fillna("").str.lower().str.strip()
dealers["is_pon_dealer"] = dealers["brand_clean"].isin(PON_BRANDS)

print(f"Pon brands found: {sorted(dealers[dealers['is_pon_dealer']]['brand_clean'].unique())}")
print(f"Pon dealer records: {dealers['is_pon_dealer'].sum():,}")
print(f"Non-Pon dealer records: {(~dealers['is_pon_dealer']).sum():,}")

Loading dealers data...
Raw dealers: 6,748 records
Pon brands found: ['cannondale', 'cervelo', 'focus', 'gazelle', 'kalkhoff', 'union']
Pon dealer records: 1,282
Non-Pon dealer records: 5,466


In [3]:
# FIXED: Preserve multi-brand information instead of aggressive deduplication
print("\nAnalyzing dealer-location relationships...")
print(f"Unique google_place_ids: {dealers['google_place_id'].nunique():,}")

# Create location-based dealer summary (preserve all brand relationships)
dealers_by_location = dealers.groupby('google_place_id').agg({
    'name': 'first',  # Take first name for location
    'google_name': 'first',  # Keep google name
    'brand_clean': lambda x: list(sorted(set(x))),  # All brands at location
    'is_pon_dealer': 'any',  # True if any Pon brand present
    'google_rating': 'first',
    'google_user_ratings_total': 'first', 
    'google_lat': 'first',
    'google_lng': 'first',
    'postal_code': 'first',
    'google_address': 'first',
    'country': 'first',
    'google_link': 'first',
    'website': 'first',
    'house_number': 'first',
    'street': 'first'
}).reset_index()

# Calculate brand metrics per location
dealers_by_location['pon_brand_count'] = dealers_by_location['brand_clean'].apply(
    lambda brands: sum(1 for b in brands if b in PON_BRANDS)
)
dealers_by_location['total_brand_count'] = dealers_by_location['brand_clean'].apply(len)
dealers_by_location['brands_sold'] = dealers_by_location['brand_clean'].apply(lambda x: ', '.join(x))

# For single-brand analysis compatibility, create primary brand column 
# (prioritize Pon brands, then alphabetical)
def get_primary_brand(brands):
    pon_brands_at_location = [b for b in brands if b in PON_BRANDS]
    if pon_brands_at_location:
        return sorted(pon_brands_at_location)[0]  # First Pon brand alphabetically
    return sorted(brands)[0]  # First non-Pon brand

dealers_by_location['brand_clean'] = dealers_by_location['brand_clean'].apply(get_primary_brand)

# Also create 'brand' column for compatibility with downstream code
dealers_by_location['brand'] = dealers_by_location['brand_clean']

print(f"After location aggregation: {len(dealers_by_location):,} unique locations")
print(f"Locations with Pon brands: {dealers_by_location['is_pon_dealer'].sum():,}")
print(f"Multi-brand locations: {(dealers_by_location['total_brand_count'] > 1).sum():,}")
print(f"Multi-Pon-brand locations: {(dealers_by_location['pon_brand_count'] > 1).sum():,}")

# Show multi-brand statistics
multi_brand_stats = dealers_by_location[dealers_by_location['total_brand_count'] > 1]
if len(multi_brand_stats) > 0:
    print(f"\n📊 Multi-brand location insights:")
    print(f"   Average brands per multi-brand location: {multi_brand_stats['total_brand_count'].mean():.1f}")
    print(f"   Locations with 2+ Pon brands: {(multi_brand_stats['pon_brand_count'] > 1).sum()}")
    
    # Show sample multi-brand locations
    print(f"\n🏪 Sample multi-brand locations:")
    sample = multi_brand_stats[['name', 'brands_sold', 'pon_brand_count']].head(3)
    for _, row in sample.iterrows():
        print(f"   {row['name']}: {row['brands_sold']} ({row['pon_brand_count']} Pon)")

# Use location-based data as dealers_unique for downstream compatibility
dealers_unique = dealers_by_location.copy()

# Also save detailed brand-location matrix for advanced analysis
print(f"\n💾 Creating detailed brand-location matrix...")
brand_location_matrix = dealers.pivot_table(
    index='google_place_id',
    columns='brand_clean',
    values='is_pon_dealer',
    aggfunc='any',
    fill_value=False
).reset_index()

print(f"Brand-location matrix: {len(brand_location_matrix)} locations × {len(brand_location_matrix.columns)-1} brands")


Analyzing dealer-location relationships...
Unique google_place_ids: 2,080
After location aggregation: 2,080 unique locations
Locations with Pon brands: 912
Multi-brand locations: 1,374
Multi-Pon-brand locations: 302

📊 Multi-brand location insights:
   Average brands per multi-brand location: 4.2
   Locations with 2+ Pon brands: 302

🏪 Sample multi-brand locations:
   Fietswinkel Gelderland: cannondale, cube, gazelle, qwic, trek (2 Pon)
   Van Der Veeken Tweewielers Etten-Leur B.V.: batavus, cortina, gazelle, giant, merida, urban_arrow (1 Pon)
   Van Der Laan Tweewielers B.V.: batavus, cortina, koga, merida, sparta (0 Pon)

💾 Creating detailed brand-location matrix...
Brand-location matrix: 2080 locations × 24 brands


In [11]:
# Extract PC4 from postal codes
print("\nProcessing postal codes...")
dealers_unique["postal_code"] = dealers_unique["postal_code"].astype(str).str.replace(" ", "", regex=False)
dealers_unique["pc4"] = dealers_unique["postal_code"].str[:4].str.extract(r"(\d{4})")[0]

# Check PC4 quality
valid_pc4 = dealers_unique["pc4"].notna()
print(f"Valid PC4 codes: {valid_pc4.sum():,}/{len(dealers_unique):,} ({valid_pc4.mean():.1%})")

# Show some examples
print("\nSample PC4 extraction:")
sample = dealers_unique[["postal_code", "pc4", "google_name", "is_pon_dealer", "brands_sold"]].head(10)
print(sample.to_string(index=False))


Processing postal codes...
Valid PC4 codes: 2,078/2,080 (99.9%)

Sample PC4 extraction:
postal_code  pc4                                                    google_name  is_pon_dealer                                                                    brands_sold
     6333BC 6333                                             Cube Store Limburg          False                                                                           cube
     6666LR 6666                                           Bike Shop Gelderland           True                                          cannondale, cube, gazelle, qwic, trek
     5025VD 5025                                              Bike Shop Ventoux          False                                                                          scott
     4876AP 4876                                 Van der Veeken Bike Specialist           True                          batavus, cortina, gazelle, giant, merida, urban_arrow
     7921GC 7921                         

In [12]:
# Save processed dealers and brand analysis data
print("\nSaving processed dealers...")

# Save main location-based dealers file (for coverage, KPIs, etc.)
dealers_unique.to_parquet(PROC / "dealers.parquet", index=False)
print(f"✅ Saved to {PROC / 'dealers.parquet'}")

# Save detailed brand-location matrix for portfolio analysis
brand_location_matrix.to_parquet(PROC / "brand_location_matrix.parquet", index=False)
print(f"✅ Saved to {PROC / 'brand_location_matrix.parquet'}")

# Save raw dealer data with brand relationships for detailed analysis
dealers.to_parquet(PROC / "dealers_all_brands.parquet", index=False)
print(f"✅ Saved to {PROC / 'dealers_all_brands.parquet'}")

# Create comprehensive brand performance summary
brand_performance_corrected = dealers[dealers['is_pon_dealer']].groupby('brand_clean').agg({
    'google_place_id': 'nunique',  # Unique locations per brand
    'name': 'count',  # Total brand relationships
    'google_rating': 'mean',
    'google_user_ratings_total': 'mean'
}).reset_index()

brand_performance_corrected.columns = ['brand', 'unique_locations', 'total_relationships', 
                                     'avg_rating', 'avg_reviews']

# Calculate performance metrics
brand_performance_corrected['market_presence'] = (
    brand_performance_corrected['unique_locations'] / dealers['google_place_id'].nunique() * 100
)

brand_performance_corrected = brand_performance_corrected.sort_values('total_relationships', ascending=False)
brand_performance_corrected.to_csv(OUT / "brand_performance_corrected.csv", index=False)
print(f"✅ Saved to {OUT / 'brand_performance_corrected.csv'}")

print(f"\n📊 CORRECTED Pon brand performance:")
print(brand_performance_corrected.to_string(index=False))

# Calculate true market share
total_dealer_relationships = len(dealers)
pon_dealer_relationships = dealers['is_pon_dealer'].sum()
true_market_share = pon_dealer_relationships / total_dealer_relationships * 100

print(f"\n📈 CORRECTED Market Metrics:")
print(f"   Total dealer relationships: {total_dealer_relationships:,}")
print(f"   Pon dealer relationships: {pon_dealer_relationships:,}")
print(f"   True Pon market share: {true_market_share:.1f}% (was 43.8%)")
print(f"   Unique dealer locations: {dealers['google_place_id'].nunique():,}")
print(f"   Multi-brand locations: {(dealers_unique['total_brand_count'] > 1).sum():,}")


Saving processed dealers...
✅ Saved to ../data/processed/dealers.parquet
✅ Saved to ../data/processed/brand_location_matrix.parquet
✅ Saved to ../data/processed/dealers_all_brands.parquet
✅ Saved to ../outputs/tables/brand_performance_corrected.csv

📊 CORRECTED Pon brand performance:
     brand  unique_locations  total_relationships  avg_rating  avg_reviews  market_presence
   gazelle               700                  701    4.470000    69.812857        33.653846
     union               247                  247    4.395082    76.426230        11.875000
  kalkhoff               191                  199    4.458081    97.520202         9.182692
cannondale                92                   92    4.564130   153.228261         4.423077
   cervelo                35                   35    4.582857    74.571429         1.682692
     focus                 8                    8    4.887500    56.250000         0.384615

📈 CORRECTED Market Metrics:
   Total dealer relationships: 6,748
   P

## 2. Demographics Data Processing

In [13]:
# Load demografie data
print("Loading demographics data...")
demo = pd.read_csv(RAW / "demografie.csv", dtype=str)
print(f"Raw demographics: {len(demo):,} records")
print(f"Columns: {list(demo.columns[:10])}...")  # First 10 columns

Loading demographics data...
Raw demographics: 4,070 records
Columns: ['Postcode-4', 'Totaal inwoners x1', 'Man x1', 'Vrouw x1', 'tot 15 jaar x1', '15 tot 25 jaar x1', '25 tot 45 jaar x1', '45 tot 65 jaar x1', '65 jaar en ouder x1', 'Geboren in Nederland met een Nederlandse herkomst %']...


In [14]:
# Clean headers (remove newlines and extra spaces)
demo.columns = demo.columns.str.strip().str.replace("\n", " ", regex=False)
print("Cleaned column headers")

# Handle CBS sentinel value -99997 and convert to numeric
print("\nProcessing numeric columns...")
numeric_cols = demo.columns[1:]  # All except first column (PC4)

for col in numeric_cols:
    # Replace commas with dots for decimal conversion
    demo[col] = demo[col].str.replace(",", ".", regex=False)
    # Convert to numeric
    demo[col] = pd.to_numeric(demo[col], errors="coerce")
    # Replace CBS sentinel -99997 with NaN
    demo[col] = demo[col].mask(demo[col] == -99997, np.nan)

print(f"Processed {len(numeric_cols)} numeric columns")

Cleaned column headers

Processing numeric columns...
Processed 37 numeric columns


In [15]:
# Rename PC4 column for consistency
if "Postcode-4" in demo.columns:
    demo = demo.rename(columns={"Postcode-4": "pc4"})
elif "PC4" in demo.columns:
    demo = demo.rename(columns={"PC4": "pc4"})
    
print(f"PC4 column found: {'pc4' in demo.columns}")
print(f"Unique PC4s: {demo['pc4'].nunique():,}")

PC4 column found: True
Unique PC4s: 4,070


In [16]:
# Extract gemeente from dealer addresses for mapping
print("\nExtracting gemeente from dealer addresses...")

# Debug: Check available columns in dealers_unique
print(f"Available columns in dealers_unique: {list(dealers_unique.columns)}")
print(f"Available columns in demo: {list(demo.columns)}")

# Check if both have pc4 columns
if 'pc4' not in dealers_unique.columns:
    print("⚠️ pc4 column not found in dealers_unique")
else:
    print(f"✅ pc4 column found in dealers_unique: {dealers_unique['pc4'].nunique()} unique values")

if 'pc4' not in demo.columns:
    print("⚠️ pc4 column not found in demo")
else:
    print(f"✅ pc4 column found in demo: {demo['pc4'].nunique()} unique values")

# Only proceed if both have pc4 columns
if 'pc4' in dealers_unique.columns and 'pc4' in demo.columns:
    # Split address and clean gemeente name
    address_parts = dealers_unique['google_address'].str.split(',')
    dealers_unique['gemeente'] = address_parts.str[-2].str.strip()
    
    # Remove postcode from gemeente (pattern: remove 4 digits + 2 letters at start)
    dealers_unique['gemeente'] = dealers_unique['gemeente'].str.replace(r'^\d{4}\s*[A-Z]{2}\s*', '', regex=True)
    dealers_unique['gemeente'] = dealers_unique['gemeente'].str.replace(' Nederland', '').str.strip()
    
    # Create PC4-gemeente mapping from dealers
    pc4_gemeente_from_dealers = dealers_unique[['pc4', 'gemeente']].dropna()
    pc4_gemeente_from_dealers = pc4_gemeente_from_dealers[pc4_gemeente_from_dealers['gemeente'] != '']
    pc4_gemeente_from_dealers = pc4_gemeente_from_dealers.drop_duplicates('pc4')
    
    print(f"Created PC4-gemeente mapping from {len(pc4_gemeente_from_dealers)} dealers")
    
    # Apply to demographics
    demo = demo.merge(pc4_gemeente_from_dealers, on='pc4', how='left')
    mapped_pct = demo['gemeente'].notna().mean()
    print(f"PC4s mapped to gemeente via dealers: {mapped_pct:.1%}")
    
    # Show sample mapping
    print("\nSample gemeente mapping:")
    sample_mapping = pc4_gemeente_from_dealers.head(10)
    print(sample_mapping.to_string(index=False))
    
else:
    print("⚠️ Skipping gemeente mapping due to missing pc4 columns")
    # Add empty gemeente column to demo for compatibility
    if 'gemeente' not in demo.columns:
        demo['gemeente'] = None


Extracting gemeente from dealer addresses...
Available columns in dealers_unique: ['google_place_id', 'name', 'google_name', 'brand_clean', 'is_pon_dealer', 'google_rating', 'google_user_ratings_total', 'google_lat', 'google_lng', 'postal_code', 'google_address', 'country', 'google_link', 'website', 'house_number', 'street', 'pon_brand_count', 'total_brand_count', 'brands_sold', 'brand', 'pc4', 'gemeente']
Available columns in demo: ['pc4', 'Totaal inwoners x1', 'Man x1', 'Vrouw x1', 'tot 15 jaar x1', '15 tot 25 jaar x1', '25 tot 45 jaar x1', '45 tot 65 jaar x1', '65 jaar en ouder x1', 'Geboren in Nederland met een Nederlandse herkomst %', 'Geboren in Nederland met een Europese herkomst (excl. Nederland) %', 'Geboren in Nederland met herkomst buiten Europa %', 'Geboren buiten Nederland met een Europese herkomst (excl. Nederland) %', 'Geboren buiten Nederland met een herkomst buiten Europa %', 'Totaal huishouden x1', 'Eenpersoons x1', 'Meerpersoons  zonder kinderen x1', 'Eenouder x1', 

## 3. Feature Engineering

In [17]:
# Define feature mapping (adjust column names to match your CSV)
print("Creating feature mapping...")
print("Available columns:", list(demo.columns))

Creating feature mapping...
Available columns: ['pc4', 'Totaal inwoners x1', 'Man x1', 'Vrouw x1', 'tot 15 jaar x1', '15 tot 25 jaar x1', '25 tot 45 jaar x1', '45 tot 65 jaar x1', '65 jaar en ouder x1', 'Geboren in Nederland met een Nederlandse herkomst %', 'Geboren in Nederland met een Europese herkomst (excl. Nederland) %', 'Geboren in Nederland met herkomst buiten Europa %', 'Geboren buiten Nederland met een Europese herkomst (excl. Nederland) %', 'Geboren buiten Nederland met een herkomst buiten Europa %', 'Totaal huishouden x1', 'Eenpersoons x1', 'Meerpersoons  zonder kinderen x1', 'Eenouder x1', 'Tweeouder x1', 'Huishoudgrootte x1', 'Totaal woning x1', 'voor 1945 x1', '1945 tot 1965 x1', '1965 tot 1975 x1', '1975 tot 1985 x1', '1985 tot 1995 x1', '1995 tot 2005 x1', '2005 tot 2015 x1', '2015 en later x1', 'Meergezins x1', 'Koopwoning %', 'Huurwoning %', 'Huurcoporatie x1', 'Niet bewoond x1', 'WOZ-waarde woning x 1 000 Euro', 'Personen met WW, Bijstand en/of AO uitkering Beneden A

In [18]:
# Map to standardized feature names
# Note: These names should match your actual CSV columns
feature_mapping = {}

# Try to find population column
pop_cols = [col for col in demo.columns if 'inwoner' in col.lower() and 'totaal' in col.lower()]
if pop_cols:
    feature_mapping['pop_total'] = pop_cols[0]
    
# Try to find household columns
hh_cols = [col for col in demo.columns if 'huishouden' in col.lower()]
hh_total_cols = [col for col in hh_cols if 'totaal' in col.lower()]
if hh_total_cols:
    feature_mapping['hh_total'] = hh_total_cols[0]
    
hh_1p_cols = [col for col in hh_cols if 'eenpersoons' in col.lower()]
if hh_1p_cols:
    feature_mapping['hh_1p'] = hh_1p_cols[0]

# Try to find age columns
age_cols = [col for col in demo.columns if 'jaar' in col.lower()]
kids_cols = [col for col in age_cols if ('15' in col or '0' in col) and 'tot' in col.lower()]
if kids_cols:
    feature_mapping['kids_0_15'] = kids_cols[0]
    
adult_cols = [col for col in age_cols if '25' in col and '45' in col]
if adult_cols:
    feature_mapping['age_25_44'] = adult_cols[0]

# Housing
koop_cols = [col for col in demo.columns if 'koopwoning' in col.lower() and '%' in col]
if koop_cols:
    feature_mapping['koop_pct'] = koop_cols[0]
    
huur_cols = [col for col in demo.columns if 'huurwoning' in col.lower() and '%' in col]
if huur_cols:
    feature_mapping['huur_pct'] = huur_cols[0]

# Density and WOZ
density_cols = [col for col in demo.columns if 'dichtheid' in col.lower()]
if density_cols:
    feature_mapping['density'] = density_cols[0]
    
woz_cols = [col for col in demo.columns if 'woz' in col.lower()]
if woz_cols:
    feature_mapping['woz'] = woz_cols[0]

print("\nFeature mapping:")
for k, v in feature_mapping.items():
    print(f"{k:12} -> {v}")
    
print(f"\nMapped {len(feature_mapping)} features")


Feature mapping:
pop_total    -> Totaal inwoners x1
hh_total     -> Totaal huishouden x1
kids_0_15    -> tot 15 jaar x1
age_25_44    -> 25 tot 45 jaar x1
koop_pct     -> Koopwoning %
huur_pct     -> Huurwoning %
density      -> Omgevingsadressendichtheid adressen/km2
woz          -> WOZ-waarde woning x 1 000 Euro

Mapped 8 features


In [23]:
# Create feature dataset with mapped columns
base_cols = ['pc4', 'gemeente']
feature_cols = [feature_mapping[k] for k in feature_mapping.keys()]

demo_feat = demo[base_cols + feature_cols].copy()

# Rename to standardized names
rename_dict = {v: k for k, v in feature_mapping.items()}
demo_feat = demo_feat.rename(columns=rename_dict)

print(f"Feature dataset: {len(demo_feat):,} rows x {len(demo_feat.columns)} columns")
print(f"Columns: {list(demo_feat.columns)}")

Feature dataset: 4,070 rows x 10 columns
Columns: ['pc4', 'gemeente', 'pop_total', 'hh_total', 'kids_0_15', 'age_25_44', 'koop_pct', 'huur_pct', 'density', 'woz']


In [25]:
# Derive ratio features
print("\nCreating ratio features...")

# Household ratios
if 'hh_1p' in demo_feat.columns and 'hh_total' in demo_feat.columns:
    demo_feat["hh_1p_pct"] = demo_feat["hh_1p"] / demo_feat["hh_total"]
    print("✅ hh_1p_pct created")

# Age ratios  
if 'kids_0_15' in demo_feat.columns and 'pop_total' in demo_feat.columns:
    demo_feat["kids_0_15_pct"] = demo_feat["kids_0_15"] / demo_feat["pop_total"]
    print("✅ kids_0_15_pct created")
    
if 'age_25_44' in demo_feat.columns and 'pop_total' in demo_feat.columns:
    demo_feat["age_25_44_pct"] = demo_feat["age_25_44"] / demo_feat["pop_total"]
    print("✅ age_25_44_pct created")

# Create income/wealth proxies from WOZ
if 'woz' in demo_feat.columns:
    woz_valid = demo_feat['woz'].notna() & (demo_feat['woz'] > 0)
    if woz_valid.sum() > 0:
        woz_min, woz_max = demo_feat.loc[woz_valid, 'woz'].min(), demo_feat.loc[woz_valid, 'woz'].max()
        demo_feat['income_norm'] = (demo_feat['woz'] - woz_min) / (woz_max - woz_min)
        print("✅ income_norm created from WOZ")

# Density normalization
if 'density' in demo_feat.columns:
    density_valid = demo_feat['density'].notna() & (demo_feat['density'] > 0)
    if density_valid.sum() > 0:
        dens_min, dens_max = demo_feat.loc[density_valid, 'density'].min(), demo_feat.loc[density_valid, 'density'].max()
        demo_feat['density_norm'] = (demo_feat['density'] - dens_min) / (dens_max - dens_min)
        print("✅ density_norm created")

print(f"\nFinal feature set: {list(demo_feat.columns)}")


Creating ratio features...
✅ kids_0_15_pct created
✅ age_25_44_pct created
✅ income_norm created from WOZ
✅ density_norm created

Final feature set: ['pc4', 'gemeente', 'pop_total', 'hh_total', 'kids_0_15', 'age_25_44', 'koop_pct', 'huur_pct', 'density', 'woz', 'kids_0_15_pct', 'age_25_44_pct', 'income_norm', 'density_norm']


In [26]:
  print(f"demo_feat columns: {list(demo_feat.columns)}")
  print(f"demo_feat shape: {demo_feat.shape}")

demo_feat columns: ['pc4', 'gemeente', 'pop_total', 'hh_total', 'kids_0_15', 'age_25_44', 'koop_pct', 'huur_pct', 'density', 'woz', 'kids_0_15_pct', 'age_25_44_pct', 'income_norm', 'density_norm']
demo_feat shape: (4070, 14)


## 4. Demographic Clustering

In [27]:
# Select clustering features (use available ones)
clustering_features = []
candidate_features = ["kids_0_15_pct", "age_25_44_pct", "koop_pct", "huur_pct", "density_norm", "income_norm"]

# Check which features are actually available
print("Checking available clustering features:")
for feat in candidate_features:
    if feat in demo_feat.columns:
        clustering_features.append(feat)
        print(f"  ✅ {feat}")
    else:
        print(f"  ❌ {feat} (not found)")
        
print(f"\nClustering features selected: {clustering_features}")

if len(clustering_features) >= 3:
    # Prepare data for clustering
    X = demo_feat[clustering_features].astype(float)
    
    # Fill NaN with median
    X_filled = X.fillna(X.median())
    
    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_filled)
    
    # K-means clustering
    print(f"\nPerforming K-means clustering (k=5) with {len(clustering_features)} features...")
    kmeans = KMeans(n_clusters=5, n_init=10, random_state=42)
    demo_feat["cluster"] = kmeans.fit_predict(X_scaled)
    
    # Cluster summary
    cluster_counts = demo_feat['cluster'].value_counts().sort_index()
    print(f"Cluster distribution:")
    for i, count in cluster_counts.items():
        print(f"  Cluster {i}: {count:,} PC4s ({count/len(demo_feat):.1%})")
    
    # Show feature importance by cluster centers
    print(f"\nCluster centers (standardized):")
    centers_df = pd.DataFrame(kmeans.cluster_centers_, columns=clustering_features)
    print(centers_df.round(2).to_string())
        
else:
    print(f"⚠️ Only {len(clustering_features)} features available (need 3+), assigning random clusters")
    np.random.seed(42)
    demo_feat["cluster"] = np.random.choice(5, size=len(demo_feat))

Checking available clustering features:
  ✅ kids_0_15_pct
  ✅ age_25_44_pct
  ✅ koop_pct
  ✅ huur_pct
  ✅ density_norm
  ✅ income_norm

Clustering features selected: ['kids_0_15_pct', 'age_25_44_pct', 'koop_pct', 'huur_pct', 'density_norm', 'income_norm']

Performing K-means clustering (k=5) with 6 features...
Cluster distribution:
  Cluster 0: 450 PC4s (11.1%)
  Cluster 1: 1,494 PC4s (36.7%)
  Cluster 2: 613 PC4s (15.1%)
  Cluster 3: 203 PC4s (5.0%)
  Cluster 4: 1,310 PC4s (32.2%)

Cluster centers (standardized):
   kids_0_15_pct  age_25_44_pct  koop_pct  huur_pct  density_norm  income_norm
0           1.74           0.41      0.39     -0.38         -0.11         0.91
1           0.01          -0.12      0.02     -0.07         -0.20        -0.39
2          -0.13           0.60     -1.40      1.41          0.83        -0.70
3          -0.94           2.62     -2.13      2.19          2.98        -0.09
4          -0.40          -0.69      0.83     -0.78         -0.58         0.48


## 5. Data Quality Checks & Exports

In [28]:
# Data quality summary
print("=== DATA QUALITY SUMMARY ===")
print(f"Demographics: {len(demo_feat):,} PC4 areas")
print(f"Dealers: {len(dealers_unique):,} unique dealers")

if 'pop_total' in demo_feat.columns:
    total_pop = demo_feat['pop_total'].sum(skipna=True)
    print(f"Total population: {total_pop:,.0f}")

print(f"\nMissing values per key column:")
key_cols = ['gemeente', 'pop_total', 'density', 'woz']
for col in key_cols:
    if col in demo_feat.columns:
        missing_pct = demo_feat[col].isna().mean()
        print(f"  {col}: {missing_pct:.1%} missing")
        
print(f"\nCluster distribution:")
print(demo_feat['cluster'].value_counts().sort_index().to_string())

=== DATA QUALITY SUMMARY ===
Demographics: 4,070 PC4 areas
Dealers: 2,080 unique dealers
Total population: 17,809,175

Missing values per key column:
  gemeente: 66.7% missing
  pop_total: 0.4% missing
  density: 0.0% missing
  woz: 0.9% missing

Cluster distribution:
cluster
0     450
1    1494
2     613
3     203
4    1310


In [29]:
# Save processed data
print("\n=== SAVING OUTPUTS ===")

# Save full demographics with features
demo_feat.to_parquet(PROC / "demografie.parquet", index=False)
print(f"✅ Saved {PROC / 'demografie.parquet'}")

# Save PC4-level KPIs with clusters for later use
demo_feat.to_csv(OUT / "kpis_by_pc4_with_clusters.csv", index=False)
print(f"✅ Saved {OUT / 'kpis_by_pc4_with_clusters.csv'}")

# Quick gemeente-level aggregation for validation
if 'gemeente' in demo_feat.columns and 'pop_total' in demo_feat.columns:
    gemeente_agg = demo_feat.groupby("gemeente", dropna=False).agg(
        pop_total=("pop_total", "sum"),
        pc4_count=("pc4", "count"),
        cluster_mode=("cluster", lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 0)
    ).reset_index()
    
    gemeente_agg.to_csv(OUT / "gemeente_summary.csv", index=False)
    print(f"✅ Saved {OUT / 'gemeente_summary.csv'}")
    print(f"   Gemeenten: {len(gemeente_agg):,}")


=== SAVING OUTPUTS ===
✅ Saved ../data/processed/demografie.parquet
✅ Saved ../outputs/tables/kpis_by_pc4_with_clusters.csv
✅ Saved ../outputs/tables/gemeente_summary.csv
   Gemeenten: 818


In [30]:
# Final summary
print("\n=== 01_DATAPREP COMPLETED ===")
print(f"📁 Dealers processed: {len(dealers_unique):,} unique")
print(f"📁 Demographics processed: {len(demo_feat):,} PC4 areas")
print(f"📁 Features created: {len([c for c in demo_feat.columns if c.endswith('_pct') or c.endswith('_norm')])}")
print(f"📁 Clusters: 5 demographic segments")
print(f"")
print(f"✅ Ready for 02_coverage.ipynb")
print(f"")
print(f"Key outputs:")
print(f"  - data/processed/dealers.parquet")
print(f"  - data/processed/demografie.parquet")
print(f"  - outputs/tables/kpis_by_pc4_with_clusters.csv")


=== 01_DATAPREP COMPLETED ===
📁 Dealers processed: 2,080 unique
📁 Demographics processed: 4,070 PC4 areas
📁 Features created: 6
📁 Clusters: 5 demographic segments

✅ Ready for 02_coverage.ipynb

Key outputs:
  - data/processed/dealers.parquet
  - data/processed/demografie.parquet
  - outputs/tables/kpis_by_pc4_with_clusters.csv
